In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [3]:
train_path = "/kaggle/input/titanic/train.csv"
test_path = "/kaggle/input/titanic/test.csv"
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [4]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
features = ["Pclass", "Sex", "SibSp", "Parch", "Fare"]
train_y = train_data["Survived"]
train_x = train_data[features]

# test_y = test_data["Survived"]
test_x = test_data[features]

**Converting gender strings using One Hot Encoding**

In [6]:
train_x = pd.get_dummies(train_x, drop_first=True, columns=["Sex"])
test_x = pd.get_dummies(test_x, drop_first=True, columns=["Sex"])

In [7]:
train_x.head()

,Pclass,SibSp,Parch,Fare,Sex_male
0,3,1,0,7.2500,1
1,1,1,0,71.2833,0
2,3,0,0,7.9250,0
3,1,1,0,53.1000,0
4,3,0,0,8.0500,1


In [8]:
test_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   SibSp     418 non-null    int64  
 2   Parch     418 non-null    int64  
 3   Fare      417 non-null    float64
 4   Sex_male  418 non-null    uint8  
dtypes: float64(1), int64(3), uint8(1)
memory usage: 13.6 KB


In [9]:
test_x["Fare"].fillna(0, inplace=True)

In [10]:
test_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   SibSp     418 non-null    int64  
 2   Parch     418 non-null    int64  
 3   Fare      418 non-null    float64
 4   Sex_male  418 non-null    uint8  
dtypes: float64(1), int64(3), uint8(1)
memory usage: 13.6 KB


In [11]:
model = RandomForestRegressor(n_estimators=100, max_depth=3, random_state=2)
model.fit(train_x, train_y)

RandomForestRegressor(max_depth=3, random_state=2)

In [12]:
prediction = model.predict(test_x)
prediction[:20]

array([0.0977712 , 0.6400969 , 0.11584447, 0.11278128, 0.53444574,
       0.11278128, 0.6400969 , 0.30460089, 0.6400969 , 0.16722495,
       0.0972498 , 0.21987372, 0.97703551, 0.19355087, 0.97703551,
       0.94763381, 0.12942793, 0.09529028, 0.58975295, 0.6400969 ])

In [13]:
prediction = np.where(prediction>0.5, 1, 0)
prediction[:20]

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1])

In [14]:
test_final = test_data["PassengerId"]
type(prediction)

numpy.ndarray

In [15]:
final_dataset = pd.DataFrame({"PassengerId": test_final[:], "Survived": prediction[:]})

In [16]:
final_dataset

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [17]:
final_dataset.to_csv('titanic_final2.csv',index=False)